In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.conv_learner import *
PATH = os.path.expanduser("~/data/dogbreed")

In [9]:
!ls {PATH}

labels.csv  test  train


In [10]:
label_csv = f'{PATH}/labels.csv'
with open(label_csv) as f: n = len(list(f)) - 1
val_idxs = get_cv_idxs(n)

In [11]:
val_idxs[:5]

array([2882, 4514, 7717,   33, 4924])

In [21]:
def get_data(f_model, sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_side_on, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test')

In [29]:
arch = resnet34
SZ = 96

In [30]:
data = get_data(arch, SZ)

In [ ]:
x, y = next(iter(data.val_dl))
# plt.imshow(data.val_ds.denorm(to_np(x))[0]);